# THIS NOTEBOOK MAINLY USED FOR THE CAPSTONE PROJECT

In [39]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
!conda install -c conda-forge folium=0.5.0 --yes
import folium 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

## Getting the url

In [8]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [9]:
response = requests.get(url)
response.status_code
#response.content

200


## web scraping using beautifulsoup package

In [10]:
soup = BeautifulSoup(response.content, 'html.parser')

# Finding the appropriate table

In [12]:
stat_table = soup.find_all('table', class_ = 'wikitable sortable')

In [13]:
len(stat_table)

1

In [14]:
stat_table = stat_table[0]

# To create the dataframe:


In [16]:
data =[]



for row in stat_table.find_all('tr'):
    col = row.find_all('td')
    if len(col) == 3:  
     #   print("...")
        data.append((col[0].text.strip(), col[1].text.strip(), col[2].text.strip()))
   
   
df = pd.DataFrame(data, columns=["PostalCode", "Borough", "Neighborhood"])
df.head(10)
#df.shape


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [17]:

print(df.shape)
#df.head()

(288, 3)


## Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
## If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [19]:
real =[]



for row in stat_table.find_all('tr'):
    col = row.find_all('td')
    if len(col) == 3:
        string = col[1].text.strip()
        
        if (string != 'Not assigned'):
            neigh = col[2].text.strip()
            if (neigh == 'Not assigned'):
                        
                real.append((col[0].text.strip(), col[1].text.strip(), col[1].text.strip()))
            else :
                real.append((col[0].text.strip(), col[1].text.strip(), col[2].text.strip()))
   
   
dfreal = pd.DataFrame(real, columns=["PostalCode", "Borough", "Neighborhood"])
dfreal.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [20]:
dfreal.shape

(211, 3)

## More than one neighborhood can exist in one postal code area. These two rows will be combined into one row with the neighborhoods separated with a comma as shown .

In [21]:
dfreal1 = dfreal.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
dfreal1


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [22]:
dfreal1.shape

(103, 3)

# Using the csv file to create dataframe:



In [23]:
path="http://cocl.us/Geospatial_data"
location_coordinates=pd.read_csv(path)
location_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [25]:
location_coordinates.shape

(103, 3)

In [29]:
location_coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)

dfreal1 = dfreal1.merge(location_coordinates, on="PostalCode", how="left")
dfreal1.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Explore and cluster the neighborhoods in Toronto
## I decided to work with only boroughs that contain the word Toronto

In [35]:
toronto_map = dfreal1[dfreal1['Borough'].str.contains('Toronto')]
toronto_map.head(10)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
49,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


In [36]:
toronto_map.shape

(38, 5)

In [42]:
latitude = 43.6532
longitude = -79.3872

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(toronto_map['Latitude'],toronto_map['Longitude'], toronto_map['Borough'],
                                          toronto_map['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
       [lat, lng],   radius=5,
         popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto